In [1]:
pip list

Package                   Version
------------------------- -----------
aiohappyeyeballs          2.4.3
aiohttp                   3.10.10
aiosignal                 1.3.1
allennlp                  2.9.3
allennlp-models           2.9.3
aspose-words              24.12.0
asttokens                 2.4.1
async-timeout             4.0.3
attrs                     24.2.0
base58                    2.1.1
blis                      0.7.11
boto3                     1.35.54
botocore                  1.35.54
cached-path               1.1.2
cachetools                5.5.0
catalogue                 2.0.10
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.4.0
click                     8.1.7
colorama                  0.4.6
comm                      0.2.2
conllu                    4.4.1
contourpy                 1.3.0
crosslingual-coreference  0.3.1
cryptography              44.0.0
cycler                    0.12.1
cymem                     2.0.8
datasets      

### **Sample data** (Nyxal's Reach; Warhammer 40,000 Homebrew)
##### **System Data**
+ **label**: Designation of System or Planet
+ **title**: Description of System of Planet
+ **value**: Approximate Size of the Planet (metric agnostic, using astronomical units)
+ **type**: *Sun*, *Planet*
+ **threat level**: *Minima*, *Minoris*, *Majoris*, *Extremis*, and *Terminus*

##### **Sector Map**
+ **source**: Origin system or planet
+ **target**: Destination system or planet
+ **weight**: Distance (metric agnostic, using astronomical units)
+ **type**: *Regular*, *Unpredictable*, *In-system*

In [1]:
import random 
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from PIL import Image
from pyvis.network import Network
import tkinter as tk
from textwrap import wrap

def random_yellow_hex():
    """Generates a random hex color code that falls within the yellow range."""

    # Generate random values for red and green components, ensuring a yellow hue
    red = random.randint(210, 255)
    green = random.randint(210, 255)

    # Keep blue component low to maintain yellow hue
    blue = random.randint(100, 255)

    # Format as hex code
    return f"#{red:02X}{green:02X}{blue:02X}"

def generate_earthy_hex_color():
    """Generates a random hex color code with an earthy tone."""

    # Define ranges for earthy RGB values
    red_range = (150, 255)
    green_range = (150, 255)
    blue_range = (75, 255)

    # Generate random RGB values within the earthy ranges
    red = random.randint(*red_range)
    green = random.randint(*green_range)
    blue = random.randint(*blue_range)

    # Convert RGB to hex code
    hex_code = "#{:02x}{:02x}{:02x}".format(red, green, blue)

    return hex_code

In [2]:
system_data = [
    
    {'label': 'Taransi', 'type': 'Sun', 'value': 0.009309477557958, 'threat level': 'Extremis', 'x': -54, 'y': 1, 'z': 13, 'description': '[ERROR - RECORDS EXPUNGED]'},
    {'label': 'Taransi I', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Extremis', 'x': -53, 'y': 2, 'z': 12, 'description': '[ERROR - RECORDS EXPUNGED]'},
    {'label': 'Taransi II', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Majoris', 'x': -54, 'y': -1, 'z': 14, 'description': '[ERROR - RECORDS EXPUNGED]'},
    {'label': 'Taransi III', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Extremis', 'x': -51, 'y': 3, 'z': 11, 'description': '[ERROR - RECORDS EXPUNGED]'},
    {'label': 'Taransi IV', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Extremis', 'x': -57, 'y': 4, 'z': 13, 'description': '[ERROR - RECORDS EXPUNGED]'},
    
    {'label': 'Aeretus', 'type': 'Sun', 'value': 0.009309477557958, 'threat level': 'Extremis', 'x': -55, 'y': -20, 'z': -3, 'description': '[ERROR - RECORDS EXPUNGED]'},
    {'label': 'Aeretus Major', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Extremis', 'x': -54, 'y': -22, 'z': -3, 'description': '[ERROR - RECORDS EXPUNGED]'},
    {'label': 'Aeretus Minor', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Majoris', 'x': -56, 'y': -24, 'z': -4, 'description': '[ERROR - RECORDS EXPUNGED]'},
    
    {'label': 'Thonelian', 'type': 'Sun', 'value': 0.009309477557958, 'threat level': 'Majoris', 'x': -39, 'y': -10, 'z': 1, 'description': "Muster for the regional war effort, military expeditions venture into the contested systems of Taransi and Aeretus."},
    {'label': 'Thonelia Major', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Majoris', 'x': -40, 'y': -13, 'z': 0, 'description': "Fortress world. The last line of defense should the Taransi and Aeretus systems fall."},
    {'label': 'Thonelia Minor', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Majoris', 'x': -38, 'y': -9, 'z': 2, 'description': "Dead world. Exterminatus by the authority of [REDACTED] in an effort to redirect the advance of a Tyranid splinter fleet; designation 'Oevid'."},
    
    {'label': 'Ledrol', 'type': 'Sun', 'value': 0.009309477557958, 'threat level': 'Minoris', 'x': -32, 'y': 10, 'z': -5, 'description': "Plagued by warp storms, Ledrol is a uniquely unstable system consisting of a dead, feral, and death world. The natives are born with unique cerise eyes and are viewed with suspicion by the rest of the sector. Rumours that the planets and their denizens are warp-touched have been dismissed by the Ordos Majoris."},
    {'label': 'Ledrol I', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Majoris', 'x': -33, 'y': 11, 'z': -6, 'description': "[ERROR - RECORDS EXPUNGED]"},
    {'label': 'Ledrol II', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minoris', 'x': -29, 'y': 9, 'z': -4, 'description': "Death world. Much of the surface is uninhabitable due to invasive parasitic flora with the capability of infecting and manipulating living mammalian hosts. Regularly scouring entire regions clean with orbital lance batteries has allowed the establishment of small settlements."},
    {'label': 'Ledrol III', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minoris', 'x': -33, 'y': 7, 'z': -5, 'description': "Feral world. Sparsely populated by a diverse set of tribal societies. Minimal strategic or material value."},
    
    {'label': 'Epestri', 'type': 'Sun', 'value': 0.009309477557958, 'threat level': 'Minoris', 'x': -12, 'y': -11, 'z': 10, 'description': "The gateway to Terra, all warp travel in and out of the sector must stop in the Epestri system."},
    {'label': "Epestri's Citadel", 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minoris', 'x': -13, 'y': -13, 'z': 10, 'description': "Fortress world, a diverse bastion of the Imperium's militant factions. Home primarily to the Anostian Imperial Guard, regionally known as 'The Arachnids'. After aiding in the world's defense, the Charnel Warhawks chapter of the Adeptus Astartes have started recruiting its denizens."},
    
    {'label': 'Anost Secundus', 'type': 'Sun', 'value': 0.009309477557958, 'threat level': 'Minoris', 'x': -18, 'y': 13, 'z': 3, 'description': "A verdant system of jungles and forests. The wealth generated by Brunaason's Haven ensures the residents of Vaustior Quintus live well within their means. Criminals, vagrants, and vagabonds should be careful, lest they be sent to the mines."},
    {'label': 'Kuraos', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minoris', 'x': -17, 'y': 14, 'z': 4, 'description': 'Feral world with dense jungles and a hot, humid atmosphere. Dangeous predatory megafauna make it too difficult to settle.'},
    {'label': "Brunaason's Haven", 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minoris', 'x': -15, 'y': 13, 'z': 2, 'description': 'Mining world home to the Vôhkyrn League of Votann. Major source of dacite in the region.'},
    {'label': 'Vaustior Quintus', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minoris', 'x': -19, 'y': 10, 'z': 3, 'description': "Civilized shrine world. Home to the Seventh Order of the Lady's Mantle, an isolationist group of Adepta Sororitas who have settled the forests beyond city walls."},
    
    {'label': 'Revilus', 'type': 'Sun', 'value': 0.009309477557958, 'threat level': 'Minima', 'x': 5, 'y': 16, 'z': 5, 'description': "Breadbasket of Nyxal's Reach, Revilus II is the sole producer of sustainable food for the sector. Agrarian workers typically reside on Revilus II, then retreat to the fortress world of Revilus III when threatened."},
    {'label': 'Revilus I', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minima', 'x': 4, 'y': 15, 'z': 6, 'description': "Forge world, responsible for much of the sector's technology. Primarily produces agricultural equipment to support Revilus II."},
    {'label': 'Revilus II', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minima', 'x': 5, 'y': 20, 'z': 4, 'description': "Agri-world, the sole producer of sustainable food for the sector. Due to uneven terrain, the vast majority of the planet's surface has been organized into paddy fields."},
    {'label': 'Revilus III', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minima', 'x': 8, 'y': 18, 'z': 5, 'description': "Fortress world, a major outpost and training ground for the Anostian regiment of the Astra Militarum."},
    
    {'label': "Nanthritium's Fall", 'type': 'Sun', 'value': 0.009309477557958, 'threat level': 'Minoris', 'x': 3, 'y': -13, 'z': -10, 'description': '[ERROR - RECORDS EXPUNGED]'},
    {'label': 'Rovora', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minima', 'x': 2, 'y': -15, 'z': -11, 'description': '[ERROR - RECORDS EXPUNGED]'},
    {'label': "Naogran's World", 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minoris', 'x': 6, 'y': -13, 'z': -9, 'description': '[ERROR - RECORDS EXPUNGED]'},
    {'label': "Hagona's Hope", 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minoris', 'x': 5, 'y': -17, 'z': -10, 'description': '[ERROR - RECORDS EXPUNGED]'},
    
    {'label': 'Verak Anost', 'type': 'Sun', 'value': 0.009309477557958, 'threat level': 'Minima', 'x': 0, 'y': 0, 'z': 0, 'description': "The capital system of Nyxal's Reach. The Veraks, a pair of densely populated hive worlds, generate the vast majority of the sector's equity through administratum labour. The most populous world, Anost, is home to the sector's menial labourers."},
    {'label': 'Incaeleum', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minima', 'x': 1, 'y': -1, 'z': 0, 'description': 'Death world. A burning wasteland home only to penal colonies.'},
    {'label': 'Verak Minor', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minima', 'x': 3, 'y': -1, 'z': 1, 'description': "Hive world and sister to Verak Major. Less populous, citizenry concentrated in towering hive cities interspersed by desolate ash wastes."},
    {'label': 'Verak Major', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minima', 'x': 2.5, 'y': -2, 'z': -1, 'description': "Hive world. The first settled world of Nyxal's Reach, with a civilization dating back to the golden age of expansion. The highborns of Verak Major serve as governors for most of the sector's worlds, exercising a great deal of autonomy within the Imperium of Man. Residing in his manor houses and citadels, the White Lion of Nyxal's Reach leverages his limitless authority to influence sector governance."},
    {'label': 'Anost', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minoris', 'x': -3, 'y': -3, 'z': -2, 'description': "Industrial hive world, the most populous in the system. The vast majority of the planet's surface is an urban sprawl, encircling a single spire, the home of Governor Berionne du Sallowlocke. Corruption and crime are widespread, with no 'underhive' to speak of. The sector's only native Astra Militarum regiment are conscripted here, an alternative to a life in the mines of Brunaason's Haven or the slag pits of Incaeleum's penal colonies."},
    {'label': 'Karkukol', 'type': 'Planet', 'value': 0.0000453148, 'threat level': 'Minima', 'x': 4, 'y': 3, 'z': 2, 'description': 'Dead world, research station. A desolate frozen tundra hiding a great many ancient secrets.'},
    
]

sector_map = [
    
    # Solar Systems
    {'source': 'Taransi', 'target': 'Taransi I', 'weight': 0.387, 'type': 'In-system'},
    {'source': 'Taransi', 'target': 'Taransi II', 'weight': 1, 'type': 'In-system'},
    {'source': 'Taransi', 'target': 'Taransi III', 'weight': 9.59, 'type': 'In-system'},
    {'source': 'Taransi', 'target': 'Taransi IV', 'weight': 25, 'type': 'In-system'},
    
    {'source': 'Aeretus', 'target': 'Aeretus Major', 'weight': 1, 'type': 'In-system'},
    {'source': 'Aeretus', 'target': 'Aeretus Minor', 'weight': 5.5, 'type': 'In-system'},
    
    {'source': 'Thonelian', 'target': 'Thonelia Major', 'weight': 1.8, 'type': 'In-system'},
    {'source': 'Thonelian', 'target': 'Thonelia Minor', 'weight': 3.9, 'type': 'In-system'},

    {'source': 'Ledrol', 'target': 'Ledrol I', 'weight': 0.387, 'type': 'In-system'},
    {'source': 'Ledrol', 'target': 'Ledrol II', 'weight': 1, 'type': 'In-system'},
    {'source': 'Ledrol', 'target': 'Ledrol III', 'weight': 9.59, 'type': 'In-system'},
    
    {'source': 'Epestri', 'target': "Epestri's Citadel", 'weight': 0.387, 'type': 'In-system'},
    
    {'source': 'Anost Secundus', 'target': 'Kuraos', 'weight': 0.387, 'type': 'In-system'},
    {'source': 'Anost Secundus', 'target': "Brunaason's Haven", 'weight': 1, 'type': 'In-system'},
    {'source': 'Anost Secundus', 'target': 'Vaustior Quintus', 'weight': 9.59, 'type': 'In-system'},

    {'source': 'Revilus', 'target': 'Revilus I', 'weight': 0.387, 'type': 'In-system'},
    {'source': 'Revilus', 'target': 'Revilus II', 'weight': 1, 'type': 'In-system'},
    {'source': 'Revilus', 'target': 'Revilus III', 'weight': 9.59, 'type': 'In-system'},
    
    {'source': "Nanthritium's Fall", 'target': 'Rovora', 'weight': 0.387, 'type': 'In-system'},
    {'source': "Nanthritium's Fall", 'target': "Naogran's World", 'weight': 1, 'type': 'In-system'},
    {'source': "Nanthritium's Fall", 'target': "Hagona's Hope", 'weight': 9.59, 'type': 'In-system'},
    
    {'source': 'Verak Anost', 'target': 'Incaeleum', 'weight': 0.15, 'type': 'In-system'},
    {'source': 'Verak Anost', 'target': 'Verak Minor', 'weight': 0.65, 'type': 'In-system'},
    {'source': 'Verak Anost', 'target': 'Verak Major', 'weight': 1, 'type': 'In-system'},
    {'source': 'Verak Anost', 'target': 'Anost', 'weight': 4, 'type': 'In-system'},
    {'source': 'Verak Anost', 'target': 'Karkukol', 'weight': 20, 'type': 'In-system'},
    
    # Interstellar Travel
    {'source': 'Taransi', 'target': 'Ledrol', 'weight': 250000, 'type': 'Unpredictable'},
    {'source': 'Taransi', 'target': 'Thonelian', 'weight': 275000, 'type': 'Regular'},
    {'source': 'Taransi', 'target': 'Epestri', 'weight': 400000, 'type': 'Unpredictable'},
    
    {'source': 'Aeretus', 'target': 'Thonelian', 'weight': 265000, 'type': 'Unpredictable'},

    {'source': 'Ledrol', 'target': 'Taransi', 'weight': 250000, 'type': 'Unpredictable'},
    {'source': 'Ledrol', 'target': 'Thonelian', 'weight': 200000, 'type': 'Unpredictable'},
    {'source': 'Ledrol', 'target': 'Epestri', 'weight': 300000, 'type': 'Unpredictable'},
    {'source': 'Ledrol', 'target': 'Anost Secundus', 'weight': 215000, 'type': 'Regular'},
    
    {'source': 'Thonelian', 'target': 'Taransi', 'weight': 275000, 'type': 'Regular'},
    {'source': 'Thonelian', 'target': 'Aeretus', 'weight': 265000, 'type': 'Regular'},
    {'source': 'Thonelian', 'target': 'Ledrol', 'weight': 200000, 'type': 'Unpredictable'},
    {'source': 'Thonelian', 'target': 'Epestri', 'weight': 225000, 'type': 'Regular'},
    
    {'source': 'Epestri', 'target': 'Taransi', 'weight': 400000, 'type': 'Unpredictable'},
    {'source': 'Epestri', 'target': 'Ledrol', 'weight': 300000, 'type': 'Unpredictable'},
    {'source': 'Epestri', 'target': 'Thonelian', 'weight': 225000, 'type': 'Regular'},
    {'source': 'Epestri', 'target': 'Revilus', 'weight': 600000, 'type': 'Unpredictable'},
    {'source': 'Epestri', 'target': 'Verak Anost', 'weight': 175000, 'type': 'Regular'},
    {'source': 'Epestri', 'target': "Nanthritium's Fall", 'weight': 275000, 'type': 'Regular'},

    {'source': 'Anost Secundus', 'target': 'Ledrol', 'weight': 215000, 'type': 'Regular'},
    {'source': 'Anost Secundus', 'target': 'Verak Anost', 'weight': 315000, 'type': 'Regular'},

    {'source': 'Revilus', 'target': 'Epestri', 'weight': 600000, 'type': 'Unpredictable'},
    {'source': 'Revilus', 'target': 'Verak Anost', 'weight': 415000, 'type': 'Regular'},
    
    {'source': "Nanthritium's Fall", 'target': 'Epestri', 'weight': 275000, 'type': 'Regular'},
    {'source': "Nanthritium's Fall", 'target': 'Verak Anost', 'weight': 350000, 'type': 'Regular'},
    
    {'source': 'Verak Anost', 'target': 'Epestri', 'weight': 175000, 'type': 'Regular'},
    {'source': 'Verak Anost', 'target': 'Anost Secundus', 'weight': 315000, 'type': 'Regular'},
    {'source': 'Verak Anost', 'target': 'Revilus', 'weight': 415000, 'type': 'Regular'},
    {'source': 'Verak Anost', 'target': "Nanthritium's Fall", 'weight': 350000, 'type': 'Regular'},
    
]

root = tk.Tk()
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

system_data = pd.DataFrame(system_data)
sector_map = pd.DataFrame(sector_map)

colors = []
shape = []
image = []
for n in system_data['type']: 
    if n=='Sun':
        colors.append("rgba(0, 0, 0, 0)")
        shape.append('circularImage')
        image.append('https://png.pngtree.com/png-clipart/20230518/ourmid/pngtree-realistic-sun-illustration-png-image_7096994.png')
    else: 
        colors.append(generate_earthy_hex_color())
        shape.append('dot')
        image.append('')
system_data['color'] = colors
system_data['shape'] = shape
system_data['image'] = image

colors = []
dashes = []
for n in sector_map['type']:
    if n=='Regular':
        colors.append('rgba(27, 235, 124, 0.7)')
        dashes.append(False)
    elif n=='Unpredictable':
        colors.append('rgba(255, 0, 132, 0.7)')
        dashes.append(True)
    else:
        colors.append('rgba(0, 0, 0, 0)')
        dashes.append(False)
sector_map['color'] = colors
sector_map['dashes'] = dashes

wrapped = []
wrapped_html = []
for n in system_data['description']:
    wrapped.append('\n'.join(wrap(n, width=50)))
    wrapped_html.append('<br>'.join(wrap(n, width=50)))
system_data['description_html'] = wrapped_html
system_data['description'] = wrapped

In [4]:
system_data.to_pickle('system_data.pkl')
sector_map.to_pickle('sector_map.pkl')

In [265]:
G = nx.from_pandas_edgelist(sector_map, 'source', 'target', edge_attr=['weight', 'type', 'color', 'dashes'])

for _, row in system_data.iterrows():
    nx.set_node_attributes(G, {row['label']: {'value': row['value'], 
                                              'type': row['type'], 
                                              'threat level': row['threat level'], 
                                              'x': row['x']*50, 
                                              'y': row['y']*-50, 
                                              'title': row['label'] + "\n" + row['description'], 
                                              'color': row['color'],
                                              'shape': row['shape'],
                                              'image': row['image'],
                                              'font': {'color': '#8bad6b', 'face': 'Serif'}
                                              }})
    
for u, v, data in G.edges(data=True):
    data['weight'] = 1 / data['weight']
            
nt = Network(str(screen_height*0.85)+'px', str(screen_width*0.95)+'px', bgcolor="#031101")
nt.from_nx(G)

for edge in nt.edges:
    edge['width'] = 2   
    
nt.set_options("""
  var options = {
    "nodes": {
      "physics": false
    },
    "layout": {
      "hierarchical": false
    }
  }
  """)
    
nt.show("nyxals_reach_descr.html", notebook=False)

nyxals_reach_descr.html


In [312]:
G = nx.from_pandas_edgelist(sector_map, 'source', 'target', edge_attr=['weight', 'type', 'color'])

for _, row in system_data.iterrows():
    nx.set_node_attributes(G, {row['label']: {'value': row['value'], 
                                              'type': row['type'], 
                                              'threat level': row['threat level'], 
                                              'x': row['x']*50, 
                                              'y': row['y']*-50, 
                                              'z': row['z']*50,
                                              'title': row['description_html'], 
                                              'color': row['color'],
                                              'shape': row['shape'],
                                              'image': row['image'],
                                              'font': {'color': '#8bad6b', 'face': 'Serif'}
                                              }})
    
for u, v, data in G.edges(data=True):
    data['weight'] = 1 / data['weight']
            
nt = Network(str(screen_height*0.85)+'px', str(screen_width*0.95)+'px', bgcolor="#031101")
nt.from_nx(G)

for edge in nt.edges:
    edge['width'] = 2   

sizes = []
for n in G.nodes:
    if G.nodes[n]['type']=='Sun':
        sizes.append(G.nodes[n]['value'] * 2000)
    else:    
        sizes.append(G.nodes[n]['value'] * 140000)
        
tmp = [{n:[G.nodes[n]['x'], G.nodes[n]['y'], G.nodes[n]['z']]} for n in G.nodes]
pos = {}
for d in tmp:
    pos.update(d)
    
colors = []
for n in G.nodes: 
    if G.nodes[n]['type']=='Sun':
        colors.append(random_yellow_hex())
    else: 
        colors.append(generate_earthy_hex_color())
        
def flatten(xss):
    return [x for xs in xss for x in xs]

edge_cols = flatten([[G.edges[edge]['color'], G.edges[edge]['color'], G.edges[edge]['color']] for edge in G.edges])

x_nodes = [G.nodes[n]['x'] for n in G.nodes]
y_nodes = [G.nodes[n]['y'] for n in G.nodes]
z_nodes = [G.nodes[n]['z'] for n in G.nodes]

edge_x = []
edge_y = []
edge_z = []
for edge in G.edges():
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])
    
edge_trace = go.Scatter3d(x=edge_x, y=edge_y, z=edge_z,
                          mode='lines',
                          line=dict(color=edge_cols,
                                    width=5),
                          opacity=0.3,
                          hoverinfo='none')

node_trace = go.Scatter3d(x=x_nodes, y=y_nodes, z=z_nodes,
                          mode='markers',
                          marker=dict(size=sizes, 
                                      color=colors),
                          hoverinfo='text',
                          text=["<b>" + str(x) + "</b> (" + str(G.nodes[x]['type']) + ")<br><br>" + G.nodes[x]['title'] + "<br><br><b>Threat:</b> <i>" + G.nodes[x]['threat level'] + "</i>" for x in G.nodes]
                          )

layout = go.Layout(scene=dict(xaxis=dict(visible=False,
                                         range=[min(x_nodes), max(x_nodes)]),
                              yaxis=dict(visible=False,
                                         range=[min(y_nodes), max(y_nodes)]),
                              zaxis=dict(visible=False,
                                         range=[min(z_nodes), max(z_nodes)])),
                   paper_bgcolor='rgba(0,0,0)',
                   plot_bgcolor='rgba(0,0,0,0)')

fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
fig.update(layout_showlegend=False) 

fig.add_layout_image(
    dict(
        source=Image.open("bg2.png"),
        xref="paper",
        yref="paper",
        x=0,
        y=1, 
        sizex=1,
        sizey=1,
        xanchor="left",  # Anchor the left side of the image
        yanchor="top",  # Anchor the top side of the image
        opacity=0.6,
        layer="below"
    )
)

fig.add_layout_image(
    dict(
        source=Image.open("bg.png"),
        xref="paper",
        yref="paper",
        x=0,
        y=1, 
        sizex=1,
        sizey=1,
        xanchor="left",  # Anchor the left side of the image
        yanchor="top",  # Anchor the top side of the image
        opacity=1,
        layer="above"
    )
)

fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)

fig.update_layout(
    width=1023,  # Width in pixels
    height=775,  # Height in pixels
    hoverlabel=dict(
        bgcolor="#000d03",
        font_size=16,
        font_family="Courier New"
    )
)

html_string = fig.to_html(full_html=False)

html_template = """
<!DOCTYPE html>
<html>
<head>
<title>Nyxal's Reach</title>
</head>
<body style="background-color:black;">
<div style="display: flex; justify-content: center;">
    {plot_div}
</div>
</body>
</html>
"""

centered_html = html_template.format(plot_div=html_string)

with open("Nyxals_Reach.html", "w", encoding="utf-8") as f:
    f.write(centered_html)